## Построение пользовательского интерфейса с помошью Gradio

[Gradio](https://www.gradio.app/) - это очень удобная библиотека, позволяющая строить простейшие пользовательские интерфейсы на Python в несколько строчек кода. Для работы с Gradio достаточно описать набор входных элементов управления (текстовая строка, слайдер и т.д.) и функцию, которая преобразует входные данные в выходные - всё остальное берёт на себя библиотека. Конечно, за такую простоту приходится расплачиваться слабыми возможностями кастомизации, но для создания простейших прототипов со стандартным дизайном Gradio подходит исключительно хорошо.

Для начала, установим библиотеку:

In [1]:
%pip install -q gradio


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


Прежде, чем продолжать выполнение, стоит перезапустить ядро Jupyter Kernel, чтобы библиотека стала видна.

## Простейший пример

Рассмотрим [стандартный пример](https://www.gradio.app/guides/quickstart) из документации Gradio, реализующий чуть усложнённый пример *Hello, world*. Мы используем простейший вариант авто-генерации интерфейса, в котором при вызове `gr.Interface` мы просто говорим, какие входные элементы управления нам необходимы (в нашем случае - текстовое поле ввода и слайдер), и интерфейс генерируется автоматически. Мы также указываем в параметре `fn=` функцию, которую необходимо вызвать для формирования ответа.

Основная проблема с построением пользовательского интрефейса в DataSphere состоит в том, что виртуальная машина, на которой выполняется Jupyter Notebook, недоступна снаружи из интернет, поскольку находится за межсетевым экраном. В то же время Gradio использует веб-интерфейс, и предполагает, что мы должны иметь возможность обратиться к компьютеру, на котором выполняется код, по веб-протоколам.

Gradio содержит встроенные средства преодоления таких проблем - возможность поделиться интерфейсом с помощью специального маршрутизатора. В этом случае мы указываем параметр `share=True` в методе `launch`, и в результате создаётся специальный прокси-сайт на серверах Gradio, который транслирует все запросы в наш код, выполняемый внутри DataSphere.

Запустите ячейку ниже и убедитесь, что веб-интерфейс станет доступен не только изнутри Jupyter Notebook, но и что вы сможете открыть появившуются ссылку в другом браузере с любого подключенного к интернет компьютера.

> ВАЖНО: Ссылка для открытия сайта из интернет будет выглядеть примерно так: `https://28fb53cd9f9d5a37f4.gradio.live`. Ссылка вида `http://127.0.0.1:7860` - это локальная ссылка, используемая в том случае, если вы запускаете проект на локальном компьютере.

In [2]:
import gradio as gr

def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)

demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://59804d1b73d790af6b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Как написано в предупреждении, такая ссылка будет доступна не более, чем 72 часа; а сам интерфейс будет доступен ещё меньшее время, пока работает виртуальная машина DataSphere. Но для простых демонстраций работоспособности моделей машинного обучения заказчику таких возможностей обычно достаточно.

## Классификация изображений 

В качестве более сложного примера рассмотрим классификацию изображений с помощью предобученной сети. В этом случае используем более сложный, но и более гибкий способ построения интерфейса - с помощью явного указания всех элементов управления, т.н. [блочный способ](https://www.gradio.app/docs/gradio/blockshttps://www.gradio.app/docs/gradio/blocks).

In [1]:
import gradio as gr
import tensorflow as tf
import numpy as np

# Load a pre-trained model
model = tf.keras.applications.MobileNetV2(weights="imagenet")

def classify_image(image):
    image = tf.image.resize(image, (224, 224))
    image = np.expand_dims(image, axis=0).copy()
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    preds = model.predict(image)
    res = tf.keras.applications.mobilenet_v2.decode_predictions(preds, top=3)[0]
    return { l : c for _,l,c in res }
    
with gr.Blocks() as iface:
    gr.Markdown("## Image Classifier")
    with gr.Row():
        inp = gr.Image()
        outp = gr.Label()
    btn = gr.Button("Run")
    btn.click(fn=classify_image, inputs=inp, outputs=outp)
    
iface.launch(share=True)

2024-09-05 06:16:29.430242: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-05 06:16:29.498614: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 06:16:30.480685: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-09-05 06:16:31.605931: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://22bbc062636722f376.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 1s 775ms/step


## Выводы

Gradio позволяет вам очень быстро создать стандартный пользовательский интерфейс, для демонстрации функциональности вашего приложения или модели машинного обучения заказчикам. Для прототипирования интерфейсов нет нужды создвать отдельные виртуальные машины, всё доступно прямо из среды DataSphere.

Впоследствии, для более долгосрочного хостинга интерфейсов можно сравнительно легко перенести код на Gradio на виртуальную машину.